# This is a notebook for querying the WB1 T&S instance for the 1.0.0 Beacon release.

-   You can run each cell individually by pressing "shift + enter".
-   For more information, questions, bugs, please contact us on Slack:
    -   https://join.slack.com/t/beacontechnic-wwa5548/shared_invite/zt-2dp1vv56r-tj_KFac0sAKNuAgUKPPDRg.


#### In order to get access to the Beacon endpoint, you need to fill in your unique personal token between the " " in the cell below.


In [1]:
Token = ""

#### Install the following packages, if you have not already installed them in your environment:

-   pip install requests
-   pip install xarray
-   pip install ipywidgets
-   pip install cartopy
-   pip install h5netcdf
-   pip install netcdf4
-   pip install scipy
-   pip install packaging


#### Import the required packages


In [2]:
import requests
import json
import xarray as xr
import datetime
import pandas as pd
import os
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt

#### Retrieve the available columns from the WB1 Merged endpoint

Swagger page here: https://beacon-wb1-ts.maris.nl/swagger/


In [3]:
responseinfo = requests.get(
    "https://beacon-wb1-ts.maris.nl/api/query/available-columns",
    headers={"Authorization": f"Bearer {Token}"},
)
params = responseinfo.json()

#### Below you can search through the available columns by entering text between the brackets of search_columns(" ").


In [4]:
def search_columns(search_term):
    search_term = search_term.lower()
    matches = [col for col in params if search_term in col.lower()]

    if matches:
        print("Matching columns:")
        for match in matches:
            print(match)
    else:
        print("No matching columns found.")


search_columns("COMMON")  # Enter your search term here

Matching columns:
COMMON_ODV_TAG
COMMON_EDMO_CODE
COMMON_TIME
COMMON_TIME_QC
COMMON_DEPTH
COMMON_DEPTH_P01
COMMON_DEPTH_P06
COMMON_DEPTH_STANDARD_NAME
COMMON_DEPTH_UNITS
COMMON_DEPTH_QC
COMMON_LATITUDE
COMMON_LATITUDE_P01
COMMON_LATITUDE_P06
COMMON_LATITUDE_STANDARD_NAME
COMMON_LATITUDE_UNITS
COMMON_LONGITUDE
COMMON_LONGITUDE_P01
COMMON_LONGITUDE_P06
COMMON_LONGITUDE_STANDARD_NAME
COMMON_LONGITUDE_UNITS
COMMON_POSITION_QC
COMMON_TEMPERATURE
COMMON_TEMPERATURE_QC
COMMON_TEMPERATURE_L22
COMMON_TEMPERATURE_P01
COMMON_TEMPERATURE_P06
COMMON_TEMPERATURE_STANDARD_NAME
COMMON_TEMPERATURE_UNITS
COMMON_SALINITY
COMMON_SALINITY_QC
COMMON_SALINITY_L22
COMMON_SALINITY_P01
COMMON_SALINITY_P06
COMMON_SALINITY_STANDARD_NAME
COMMON_SALINITY_UNITS
COMMON_PLATFORM_L06
COMMON_TEMPERATURE_L05
COMMON_TEMPERATURE_L35
COMMON_SALINITY_L05
COMMON_SALINITY_L35


#### This will create the query body based on your input parameters, you can add other "query_parameters" and "filters" to suit your needs.

-   For more query examples and explanations, you can take a look at https://maris-development.github.io/beacon/.


In [5]:
query = {
    "query_parameters": [
        {
            "column_name": "COMMON_TEMPERATURE",
        },
        {
            "column_name": "COMMON_TEMPERATURE_QC",
        },
        {
            "column_name": "COMMON_TEMPERATURE_L05",
        },
        {
            "column_name": "COMMON_TEMPERATURE_L22",
        },
        {"column_name": "COMMON_TEMPERATURE_L35"},
        {
            "column_name": "COMMON_TEMPERATURE_P01",
        },
        {
            "column_name": "COMMON_TEMPERATURE_P06",
        },
        {"column_name": "COMMON_TIME", "alias": "TIME-Marker"},
        {"function": "to_timestamp_nanos", "args": [
            "COMMON_TIME"], "alias": "TIME"},
        {"column_name": "COMMON_TIME_QC", "alias": "TIME_QC"},
        {"column_name": "COMMON_DEPTH"},
        {"column_name": "COMMON_DEPTH_QC"},
        {"column_name": "COMMON_LONGITUDE"},
        {"column_name": "COMMON_LATITUDE"},
        {"column_name": "COMMON_POSITION_QC"},
        {"column_name": "COMMON_EDMO_CODE"},
        {"column_name": "SOURCE_BDI"},
        {"column_name": "SOURCE_BDI_DATASET_ID"},
    ],
    "filters": [
        {
            "for_query_parameter": "TIME",
            "min": "2003-01-01T00:00:00",
            "max": "2005-12-01T00:00:00",
        },
        {"for_query_parameter": "COMMON_LONGITUDE", "min": -8, "max": 37},
        {"for_query_parameter": "COMMON_LATITUDE", "min": 30, "max": 46},
        {
            "is_not_null": {"for_query_parameter": "COMMON_TEMPERATURE"},
        },
    ],
    "output": {"format": "parquet"},
}

#### This is the post request that is sent to Beacon with the above specified body.


In [6]:
response = requests.post(
    "https://beacon-wb1-ts.maris.nl/api/query",
    json.dumps(query),
    headers={"Authorization": f"Bearer {Token}", "Content-type": "application/json"},
)

if response.status_code == 204:
    print(
        "No data has been found for your query, please update your input fields above and run the notebook again."
    )
elif response.status_code != 200:
    print(response.text)

In [7]:
%pip install --upgrade pyarrow

if not os.path.exists("./Beacon_V1.0.0_Output"):
    os.makedirs("Beacon_V1.0.0_Output")

open(
    f"./Beacon_V1.0.0_Output/WB1-Merged.parquet",
    "wb",
).write(response.content)
df = pd.read_parquet(f"./Beacon_V1.0.0_Output/WB1-Merged.parquet")
# df = df.set_index("TIME").sort_index()
df


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


,COMMON_TEMPERATURE,COMMON_TEMPERATURE_QC,COMMON_TEMPERATURE_L05,COMMON_TEMPERATURE_L22,COMMON_TEMPERATURE_L35,COMMON_TEMPERATURE_P01,COMMON_TEMPERATURE_P06,TIME-Marker,TIME,TIME_QC,COMMON_DEPTH,COMMON_DEPTH_QC,COMMON_LONGITUDE,COMMON_LATITUDE,COMMON_POSITION_QC,COMMON_EDMO_CODE,SOURCE_BDI,SOURCE_BDI_DATASET_ID
0,17.013599,1,None,None,None,SDN:P01::TEMPPR01,SDN:P06::UPAA,2004-05-16 05:49:51,2004-05-16 05:49:51,None,1.984632,1,11.585667,40.192665,None,None,BEACON-WOD,wod_011310808O
1,17.032101,1,None,None,None,SDN:P01::TEMPPR01,SDN:P06::UPAA,2004-05-16 05:49:51,2004-05-16 05:49:51,None,2.976941,1,11.585667,40.192665,None,None,BEACON-WOD,wod_011310808O
2,16.825399,1,None,None,None,SDN:P01::TEMPPR01,SDN:P06::UPAA,2004-05-16 05:49:51,2004-05-16 05:49:51,None,3.969244,1,11.585667,40.192665,None,None,BEACON-WOD,wod_011310808O
3,16.538799,1,None,None,None,SDN:P01::TEMPPR01,SDN:P06::UPAA,2004-05-16 05:49:51,2004-05-16 05:49:51,None,4.961544,1,11.585667,40.192665,None,None,BEACON-WOD,wod_011310808O
4,16.290300,1,None,None,None,SDN:P01::TEMPPR01,SDN:P06::UPAA,2004-05-16 05:49:51,2004-05-16 05:49:51,None,5.953838,1,11.585667,40.192665,None,None,BEACON-WOD,wod_011310808O
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14347105,14.486001,1,None,None,None,SDN:P01::TEMPPR01,SDN:P06::UPAA,2005-02-01 23:46:05,2005-02-01 23:46:05,1,139.274685,None,15.960100,37.818120,1,1054,BEACON-CORA-PR,2005/CO_DMQCGL01_20050201_PR_GL.nc
14347106,14.473001,1,None,None,None,SDN:P01::TEMPPR01,SDN:P06::UPAA,2005-02-01 23:46:05,2005-02-01 23:46:05,1,44.371415,None,15.960100,37.818120,1,1054,BEACON-CORA-PR,2005/CO_DMQCGL01_20050201_PR_GL.nc
14347107,14.491001,1,None,None,None,SDN:P01::TEMPPR01,SDN:P06::UPAA,2005-02-01 23:46:05,2005-02-01 23:46:05,1,130.714768,None,15.960100,37.818120,1,1054,BEACON-CORA-PR,2005/CO_DMQCGL01_20050201_PR_GL.nc
14347108,14.490001,1,None,None,None,SDN:P01::TEMPPR01,SDN:P06::UPAA,2005-02-01 23:46:05,2005-02-01 23:46:05,1,145.741512,None,15.960100,37.818120,1,1054,BEACON-CORA-PR,2005/CO_DMQCGL01_20050201_PR_GL.nc


In [8]:
import xarray as xr

xr = df.to_xarray()
xr.to_netcdf(f"./Beacon_V1.0.0_Output/WB1-Merged.nc")